<a href="https://colab.research.google.com/github/Mr94t3z/music-genre-classification/blob/main/Mengklasifikasikan_Genre_Musik_Yang_Berbeda_Dengan_Metode_Multiclass_Support_Vector_Machines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center> <b> "Mengklasifikasikan Genre Musik yang berbeda dengan Metode Multiclass Support Vector Machines, K-Means Clustering, K-Nearest Neighbors (KNN) dan Convolutional Neural Network (CNN)" </b> </center>



<br>
<b>
<table align = "left">
  <tr>
    <td>Nama</td>
    <td>: Muhamad Taopik</td>
  </tr>
  <tr>
    <td>NIM</td>
    <td>: 1197050081</td>
  </tr>
  <tr>
    <td>Kelas</td>
    <td>: IF C</td>
  </tr>
</table>
</br>



---


# **1. Install Library**

Berikut ini list library yang dibutuhkan.


---



In [ ]:
# Install library 'python_speech_features' terlebih dahulu
!pip install pip install python_speech_features

In [ ]:
# Library yang digunakan
from python_speech_features import mfcc
import scipy.io.wavfile as wav
import numpy as np

from tempfile import TemporaryFile
import os
import pickle
import random 
import operator

import math
import numpy as np

In [ ]:
# Tentukan fungsi untuk mendapatkan jarak antara feature vectors dan cari neighbors:
def getNeighbors(trainingSet, instance, k):
    distances = []
    for x in range (len(trainingSet)):
        dist = distance(trainingSet[x], instance, k )+ distance(instance, trainingSet[x], k)
        distances.append((trainingSet[x][2], dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(k):
        neighbors.append(distances[x][0])
    return neighbors

In [ ]:
# Identifikasi neightbors terdekat:
def nearestClass(neighbors):
    classVote = {}

    for x in range(len(neighbors)):
        response = neighbors[x]
        if response in classVote:
            classVote[response]+=1 
        else:
            classVote[response]=1

    sorter = sorted(classVote.items(), key = operator.itemgetter(1), reverse=True)
    return sorter[0][0]

In [ ]:
# Tentukan fungsi untuk evaluasi model:
def getAccuracy(testSet, predictions):
    correct = 0 
    for x in range (len(testSet)):
        if testSet[x][-1]==predictions[x]:
            correct+=1
    return 1.0*correct/len(testSet)



---


# **2. Dataset GTZAN Genre Collection**
Dataset nya bisa di download melalui [link](http://marsyas.info/downloads/datasets.html) ini.

**Catatan**: _Jika ingin menjalankan program ini harus mengunduh terlebih dahulu dataset file genre keseluruhan tersebut melalui link diatas serta konfigurasi file folder pada Google Drive._

---

In [ ]:
# Konfigurasi direktori file Dataset Genre Musik yang diambil dari Google Drive saya
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# Ekstrak fitur dari kumpulan data dan buang fitur ini ke dalam file .dat biner “my.dat”:
directory = "/content/drive/MyDrive/Colab Notebooks/dataset/genres/"
f= open("my.dat" ,'wb')
i=0

for folder in os.listdir(directory):
    i+=1
    if i==11 :
        break   
    for file in os.listdir(directory+folder):
      try:
        (rate,sig) = wav.read(directory+folder+"/"+file)
        mfcc_feat = mfcc(sig,rate ,winlen=0.020, appendEnergy = False)
        covariance = np.cov(np.matrix.transpose(mfcc_feat))
        mean_matrix = mfcc_feat.mean(0)
        feature = (mean_matrix , covariance , i)
        pickle.dump(feature , f)
      except Exception as e:
        print('Got an exception: ', e, ' in folder: ', folder, ' filename: ', file)
f.close()

In [ ]:
# Latih dan uji pemisahan pada dataset:
dataset = []
def loadDataset(filename , split , trSet , teSet):
    with open("my.dat" , 'rb') as f:
        while True:
            try:
                dataset.append(pickle.load(f))
            except EOFError:
                f.close()
                break  

    for x in range(len(dataset)):
        if random.random() <split :      
            trSet.append(dataset[x])
        else:
            teSet.append(dataset[x])  

trainingSet = []
testSet = []
loadDataset("my.dat" , 0.66, trainingSet, testSet)



---


# **3. Prediksi menggunakan K-Nearest Neighbors (KNN)**
Buat prediksi menggunakan K-Nearest Neighbors (KNN) dan dapatkan akurasi pada data uji.

---



In [ ]:
# Fungsi distance
def distance(instance1 , instance2 , k ):
    distance =0 
    mm1 = instance1[0] 
    cm1 = instance1[1]
    mm2 = instance2[0]
    cm2 = instance2[1]
    distance = np.trace(np.dot(np.linalg.inv(cm2), cm1)) 
    distance+=(np.dot(np.dot((mm2-mm1).transpose() , np.linalg.inv(cm2)) , mm2-mm1 )) 
    distance+= np.log(np.linalg.det(cm2)) - np.log(np.linalg.det(cm1))
    distance-= k
    return distance

In [ ]:
# Akurasi pada yang didapat data uji:
leng = len(testSet)
predictions = []
for x in range (leng):
    predictions.append(nearestClass(getNeighbors(trainingSet ,testSet[x] , 5))) 

accuracy1 = getAccuracy(testSet , predictions)
print("Akurasi:", "\033[1m", accuracy1, "\033[0m")

Akurasi:  0.6804733727810651 




---



# **4. Uji Pengklasifikasian dengan file Audio yang ada pada Google Drive**
Untuk uji coba audio bisa mengunduh-nya dari [sini](https://uweb.engr.arizona.edu/~429rns/audiofiles/audiofiles.html).


---




In [ ]:
# Uji coba audio pada lokasi folder Google Drive saya
lokasi_folder = "/content/drive/MyDrive/Colab Notebooks/dataset/audio-test/"
test_file = lokasi_folder + "test.wav"

In [ ]:
(rate, sig) = wav.read(test_file)
mfcc_feat = mfcc(sig, rate, winlen=0.020, appendEnergy=False)
covariance = np.cov(np.matrix.transpose(mfcc_feat))
mean_matrix = mfcc_feat.mean(0)
feature = (mean_matrix, covariance, i)

In [ ]:
# Lokasi default pada drive saya
from collections import defaultdict
results = defaultdict(int)

directory = "/content/drive/MyDrive/Colab Notebooks/dataset/genres/"

i = 1
for folder in os.listdir(directory):
    results[i] = folder
    i += 1

In [ ]:
# Hasil prediksi genre musik
pred = nearestClass(getNeighbors(dataset, feature, 5))
print("Musik tersebut termasuk jenis genre:", "\033[1m" + results[pred] + "\033[0m" + ".")

Musik tersebut termasuk jenis genre: classical.




---


<center> <b> <h1> SELESAI! </1> </b> </center>


---

